## 最初の大きなプロジェクト - 専門的にあなた！

###および、ツールの使用。

###しかし、最初：プッシュオーバーの紹介

プッシュオーバーは、プッシュ通知を携帯電話に送信するための気の利いたツールです。

セットアップとインストールは非常に簡単です！

https://pushover.net/にアクセスして、右上の「ログインまたはサインアップ」をクリックして無料のアカウントにサインアップし、APIキーを作成します。

サインアップしたら、ホーム画面で「アプリケーション/APIトークンの作成」をクリックし、名前（エージェントなど）を指定し、[アプリケーションの作成]をクリックします。

次に、 `.env`ファイルに2行を追加します。

pushover_user = _プッシュオーバーホーム画面の右上にあるキーを入力し、おそらくu_で始まる  
pushover_token = _キーを入力して、エージェント（または何でも）という新しいアプリケーションをクリックして、おそらくA_で始まるときにキーを入力します

`.env`ファイルを保存し、` load_dotenv（override = true） `を保存した後、環境変数を設定することを忘れないでください。

最後に、「電話、タブレット、またはデスクトップの追加」をクリックして、電話にインストールします。

In [ ]:
# 輸入

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
# 通常のスタート

load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# プッシュオーバー用

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

if pushover_user:
    print(f"Pushover user found and starts with {pushover_user[0]}")
else:
    print("プッシュオーバーユーザーが見つかりません")

if pushover_token:
    print(f"Pushover token found and starts with {pushover_token[0]}")
else:
    print("プッシュオーバートークンが見つかりません")

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [ ]:
push("HEY!!")

In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": record_user_details_json},
        {"type": "function", "function": record_unknown_question_json}]

In [ ]:
tools

In [ ]:
# この関数は、ツール呼び出しのリストを取得し、それらを実行できます。これはIFステートメントです!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # 大きな声明!!!

        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)

        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

In [ ]:
# これは、IFステートメントを回避するよりエレガントな方法です。

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Ed Donner"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n# ＃summary：\ n {summary} \ n \ n ## linkedinプロファイル：\ n {linkedin} \ n \ n "
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:

        # これはLLMへの呼び出しです - 私たちがツールJSONを渡すことを見てください

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # LLMがツールを呼び出したい場合は、それを行います！
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

##そして今展開用

このコードは「app.py」にあります

Face Spacesに展開します。

開始する前に：「me」ディレクトリでファイルを更新することを忘れないでください - あなたのLinkedInプロファイルとsummary.txt-あなたについて話すように！また、 `self.name =" ed donner "` in `app.py` ..も変更します。  

また、1_Foundationsディレクトリ内にreadmeファイルがないことを確認してください。ある場合は、削除してください。展開プロセスは、このディレクトリに新しいreadmeファイルを作成します。

1. https://huggingface.coにアクセスして、アカウントを設定します  
2。右上のアバターメニューから、アクセストークンを選択します。 「新しいトークンの作成」を選択します。書き込み許可を与えます - 書き込み許可が必要です！新しいキーの記録を保持してください。  
3。ターミナルでは、「UVツールインストール」 'Huggingface_hub [cli]' `を実行するには、Huggingfaceツールをインストールするには、「HF Authログイン」にキーを使用してコマンドラインでログインします。その後、「hf auth whoami`を実行して、ログインしていることを確認してください  
4.新しいトークンを取り、.envファイルに追加してください： `hf_token = hf_xxx`
5。1_Foundationsフォルダーから、入力： `UV Run Gradio Deploy` 
6.その指示に従ってください：「career_conversation」に名前を付け、app.pyを指定し、ハードウェアとしてcpu-basicを選択し、秘密を提供する必要があると言い、openai apiキー、プッシュオーバーユーザー、トークンを提供し、githubアクションに「いいえ」と言います。  

これらのヒントをありがとう、ロバート、ジェームズ、マーティンズ、アンドラス、プリヤに感謝します。  
次の2つのセクションをお読みください - 秘密を変更する方法、およびスペースを再配置する方法（この1_Foundationsディレクトリで作成されるreadme.mdを削除する必要がある場合があります）。

####これらの秘密の詳細：

これらの秘密で何が起こっているかに混乱している場合：それぞれの秘密のキー名と価値を入力することを望んでいます。  
`openai_api_key`  
に続く：  
`sk-proj -...`  

そして、あなたがこのように秘密を設定したくない場合、または何かがうまくいかない場合、それは問題ありません - あなたは後であなたの秘密を変えることができます：  
1。HuggingfaceのWebサイトにログインします  
2。右上のアバターメニューからプロフィール画面に移動します  
3.展開したスペースを選択します  
4.右上の設定ホイールをクリックします  
5.下にスクロールして、秘密（変数と秘密のセクション）を変更したり、スペースを削除したりできます。

####そして今あなたは展開する必要があります！

すべてを完全に交換し、キーから再び開始する場合は、この1_Foundationsフォルダーで作成されたreadme.mdを削除する必要がある場合があります。

これが私のものです：https：//huggingface.co/spaces/ed-donner/career_conversation

学生がどのように彼らが彼らの国の大統領になることができるかを尋ねたというプッシュ通知を受け取りました😂😂

展開の詳細については：

https://www.gradio.app/guides/sharing-your-app#hosting-on-hfスペース

将来あなたのスペースを削除するには：  
1。ログインHuggingface
2。アバターメニューから、プロファイルを選択します
3.スペース自体をクリックして、右上の設定ホイールを選択します
4.下部の削除セクションにスクロールします
5。


<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/exercise.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃ff7800;">演習</h2>
            <Span style = "color：＃ff7800;">•何よりもまず、これを自分で展開してください！それは本当の、貴重なツールです - 将来の履歴書.. <br/>
            •次に、リソースを改善します - あなた自身についてより良いコンテキストを追加します。ラグを知っている場合は、あなたについての知識ベースを追加してください。<br/>
            •より多くのツールを追加してください！ LLMが読み取りと書き込みできる一般的なQ＆Aを備えたSQLデータベースを使用することができますか？<br/>
            •最後のラボから評価者を持ち込み、他のエージェントパターンを追加します。
            </span>
        </td>
    </tr>
</table>

<テーブルスタイル= "マージン：0;テキストアライグ：左;幅：100％">
    <tr>
        <td style = "width：150px; height：150px; vertical-align：middle;">
            <img src = "../ assets/business.png" width = "150" height = "150" style = "display：block;" />
        </td>
        <td>
            <h2 style = "color：＃00bfff;">商業的影響</h2>
            <Span style = "color：＃00bfff;">明白な（あなたのキャリアの分身）とは別に、これはドメインの専門知識と現実世界と対話する能力を持つAIアシスタントが必要な状況でビジネスアプリケーションを持っています。
            </span>
        </td>
    </tr>
</table>